In [ ]:

import os
mask_generator = SamAutomaticMaskGenerator(sam, min_mask_region_area=400)

def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

%matplotlib inline

def compare_images(path1, path2, predictor, mask_generator):
    image_original = cv2.imread(path1)
    image_original = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB)

    image_modified  = cv2.imread(path2)
    image_modified = cv2.cvtColor(image_modified, cv2.COLOR_BGR2RGB)

    predictor.set_image(image_original)

    predictor.set_image(image_original)

    masks = mask_generator.generate(image_original)

    masks = sorted(masks, key=(lambda x: x['area']), reverse=True)[0:5]

    centers = []
    for mask in masks:
        indices = np.argwhere(mask["segmentation"])
        center = np.mean(indices, axis=0)
        if mask["segmentation"][int(center[0])][int(center[1])] == 1:
            centers.append(center)


    centers = np.stack(centers, axis=0)

    masks_scores_1 = [predictor.predict(
        point_coords=np.array([center]),
        point_labels=np.array([1]),
        multimask_output=False,
        return_logits=False,
        ) for center in centers]
    
    # for mask in masks_scores_1:
    #     print(mask[0])
    #     print(np.sum(mask[0]))
    
    # masks, scores, logits = zip(*masks_scores_1)

    mask1 = [(i,mask[0]) for (i,mask) in enumerate(masks_scores_1) if np.sum(mask[0]) > 500]
    print(mask1)
    
    if len(mask1) == 0:
        return -1.0
    else:
        mask1 = mask1[0]

    
    center = centers[mask1[0]]

    mask1 = mask1[1]

    # for mask in masks:
    #     print(mask)
    #     print(np.sum(mask))

    # masks, scores, logits = predictor.predict(
    # point_coords=centers,
    # point_labels=[1] * len(centers),
    # multimask_output=True,
    # return_logits=False,
    # )

    # print(len(masks))

    predictor.set_image(image_modified)

    mask2 = predictor.predict(
        point_coords=np.array([center]),
        point_labels=np.array([1]),
        multimask_output=False,
        return_logits=False,
        )
    
    mask2, score2, logits2 = mask2
    
    return np.sum(np.logical_and(mask1, mask2)) / np.sum(np.logical_or(mask1, mask2))

# compare_images("processed/good/000000029393.jpg", "processed/blur/JPEG/000000029393.jpg", predictor, mask_generator)


 ## Example image

In [ ]:
def main():
    # Check if the correct number of command-line arguments is provided
    if len(sys.argv) != 3:
        print("Usage: python script.py arg1 arg2")
        sys.exit(1)

    # Read the command-line arguments
    arg1 = sys.argv[1]
    arg2 = sys.argv[2]

    files = os.listdir(arg1)

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    sam_checkpoint = "sam_vit_b_01ec64.pth"
    model_type = "vit_b"

    sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
    sam.to(device=device)

    predictor = SamPredictor(sam)
    mask_generator = SamAutomaticMaskGenerator(sam)

    ious = []

    for filename in files[:10]:
        iou = compare_images(os.path.join(arg1, filename), os.path.join(arg2, filename), predictor, mask_generator)
    

if __name__ == "__main__":
    main()

In [ ]:
arg1 = "processed/good"
arg2 = "processed/blur_small/JPEG"

files = os.listdir(arg1)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

sam_checkpoint = "sam_vit_b_01ec64.pth"
model_type = "vit_b"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)
mask_generator = SamAutomaticMaskGenerator(sam)

ious = []

for filename in files[:5]:
    iou = compare_images(os.path.join(arg1, filename), os.path.join(arg2, filename), predictor, mask_generator)
    ious.append(iou)

In [ ]:
sorted(ious)
plt.plot(sorted(ious))

In [ ]:
def compare_images_manual(path1, path2, predictor, mask_generator):
    image_original = cv2.imread(path1)
    image_original = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB)

    image_modified  = cv2.imread(path2)
    image_modified = cv2.cvtColor(image_modified, cv2.COLOR_BGR2RGB)

    predictor.set_image(image_original)

    predictor.set_image(image_original)

    masks = mask_generator.generate(image_original)

    masks = sorted(masks, key=(lambda x: x['area']), reverse=True)[0:3]

    centers = []
    for mask in masks:
        indices = np.argwhere(mask["segmentation"])
        center = np.mean(indices, axis=0)
        if mask["segmentation"][int(center[0])][int(center[1])] == 1:
            centers.append(center)

    # print(centers)

    centers = np.stack(centers, axis=0)

    print(centers)

    # for mask in masks:


    #     plt.figure(figsize=(20,20))
    #     plt.imshow(image_original)
    #     show_anns([mask])
    #     plt.axis('off')
    #     plt.show()

    # centers = np.array(centers[0:1])

    #show_mask(masks[0]["segmentation"])

    masks_scores_1 = [predictor.predict(
        point_coords=np.array([center]),
        point_labels=np.array([1]),
        multimask_output=False,
        return_logits=False,
        ) for center in centers]
    
    for mask in masks_scores_1:
        print(mask[0])
        print(np.sum(mask[0]))
    
    # masks, scores, logits = zip(*masks_scores_1)

    mask1 = [(i,mask[0]) for (i,mask) in enumerate(masks_scores_1) if np.sum(mask[0]) > 2500][0]
    center = centers[mask1[0]]

    mask1 = mask1[1]

    # for mask in masks:
    #     print(mask)
    #     print(np.sum(mask))

    # masks, scores, logits = predictor.predict(
    # point_coords=centers,
    # point_labels=[1] * len(centers),
    # multimask_output=True,
    # return_logits=False,
    # )

    # print(len(masks))

    predictor.set_image(image_modified)

    mask2 = predictor.predict(
        point_coords=np.array([center]),
        point_labels=np.array([1]),
        multimask_output=False,
        return_logits=False,
        )
    
    mask2, score2, logits2 = mask2

    # masks2, scores2, logits2 = predictor.predict(
    # point_coords=centers,
    # point_labels=[1] * len(centers),
    # multimask_output=True,
    # return_logits=False,
    # )



    # for (mask1,mask2) in zip(masks,masks2):
    #     print(np.sum(np.logical_and(mask1, mask2)) / np.sum(np.logical_or(mask1, mask2)))

    # for (mask1,mask2) in zip(masks,masks2):
    print(np.sum(np.logical_and(mask1, mask2)) / np.sum(np.logical_or(mask1, mask2)))


    # for i, (mask, mask2) in enumerate(zip(masks, masks2)):
    plt.figure(figsize=(10,10))
    plt.imshow(image_original)
    
    show_mask(mask1, plt.gca())
    # show_points(centers, [1] * len(centers), plt.gca())
    # plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
    plt.axis('off')
    plt.show()

    plt.figure(figsize=(10,10))
    plt.imshow(image_modified)
    
    show_mask(mask2, plt.gca())
    # show_points(centers, [1] * len(centers), plt.gca())
    # plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
    plt.axis('off')
    plt.show()

    # print(np.sum(mask))


        # plt.savefig('temp.png')
    
    return np.sum(np.logical_and(mask1, mask2)) / np.sum(np.logical_or(mask1, mask2))

In [ ]:
compare_images_manual(os.path.join(arg1, "000000060899.jpg"), os.path.join(arg2, "000000060899.jpg"), predictor, mask_generator)

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import sys
from torch.autograd import Variable
from typing import Optional, Tuple
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor, SamAutomaticMaskGenerator
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#sam_checkpoint = "sam_vit_h_4b8939.pth"
sam_checkpoint = "sam_vit_b_01ec64.pth"
model_type = "vit_b"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

In [ ]:
predictor.predict()

In [ ]:
def show_mask(mask, ax, random_color=False,target_size=None):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    # resize the mask to target_size (if provided) (H,W) -> (target_size, target_size)
    if target_size:
        mask_image = cv2.resize(mask_image, (target_size, target_size))
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))